In [25]:
import json
import numpy as np

data = json.load(open('../data/datasets/conll04/conll04_train_dev_clean.json'))
# data = json.load(open('../data/datasets/scierc/scierc_train_dev.json'))

rel_head_dict = {}
rel_tail_dict = {}
rels = set()
ents = set()

for cur_d in data:
    for ent in cur_d['entities']:
        ents.add(ent['type'])

    for rel in cur_d['relations']:
        rels.add(rel['type'])
        if rel['type'] not in rel_head_dict:
            rel_head_dict[rel['type']] = {}
        if rel['type'] not in rel_tail_dict:
            rel_tail_dict[rel['type']] = {}
        
        head_ent_type = cur_d['entities'][rel['head']]['type']
        tail_ent_type = cur_d['entities'][rel['tail']]['type']

        if head_ent_type not in rel_head_dict[rel['type']]:
            rel_head_dict[rel['type']][head_ent_type] = 0
        rel_head_dict[rel['type']][head_ent_type] += 1
        if tail_ent_type not in rel_tail_dict[rel['type']]:
            rel_tail_dict[rel['type']][tail_ent_type] = 0
        rel_tail_dict[rel['type']][tail_ent_type] += 1
    
rels = list(rels)
ents = list(ents)

rel_head_prior = np.zeros((len(rels), len(ents)), np.float32)
rel_tail_prior = np.zeros((len(rels), len(ents)), np.float32)

print(len(rels), len(ents), rel_head_prior.shape)

for rel_idx, rel in enumerate(rels):
    cur_rel_head_sum = sum(rel_head_dict[rel].values())
    for ent_idx, ent in enumerate(ents):
        rel_head_prior[rel_idx, ent_idx] = rel_head_dict[rel].get(ent, 0) / cur_rel_head_sum

for rel_idx, rel in enumerate(rels):
    cur_rel_tail_sum = sum(rel_tail_dict[rel].values())
    for ent_idx, ent in enumerate(ents):
        rel_tail_prior[rel_idx, ent_idx] = rel_tail_dict[rel].get(ent, 0) / cur_rel_tail_sum

5 4 (5, 4)


In [26]:
print(rels)
print(ents)
print(rel_head_prior)
print(rel_tail_prior)

['Kill', 'Work_For', 'Live_In', 'OrgBased_In', 'Located_In']
['Org', 'Loc', 'Peop', 'Other']
[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]
